In [1]:
import numpy as np
# import matplotlib.pyplot as plt
import mujoco 
# import mujoco.viewer
import time
# import desired_kinematic as dsk
# import compute_activation as ca
# import force_length_velocity_functions as flv
import get_activations
import plot_functions

### Compute muscle activations based on desired joint posture using quadruped in air model

In [ ]:

path_to_model = "../Working_Folder/tendon_quadruped_ws_inair.xml"

activation_folder = "../activation_data_files" 

for omega in np.round(np.arange(0.01, 1.21, 0.01),2):  # Increment omega from 0.1 to 1.2 (inclusive)
    try: 
        activation_file, desired_qpos, desired_qvel, desired_qacc = get_activations.compute_and_save_activations(
            path_to_model, 
            omega, 
            dt=0.005, 
            duration_in_seconds=10,
            activation_folder=activation_folder)

        muscle_activations = np.loadtxt(activation_file)
        print(f"Simulated omega: {omega}, Activation file: {activation_file}")

    except Exception as e:
        print(f"An error occurred while simulating omega {omega}: {e}")
        continue


In [ ]:
plot_functions.plot_muscle_activations(muscle_activations)

In [ ]:
plot_functions.plot_joint_state(desired_qpos)

### Run simulation of quadruped in air using found activations

In [250]:
path_to_model = "../Working_Folder/tendon_quadruped_ws_inair.xml"
activation_folder = "../activation_data_files" 

model = mujoco.MjModel.from_xml_path(path_to_model)
data = mujoco.MjData(model)

omega = 1.2
activation_file = f"{activation_folder}/activation_{omega}.txt"
muscle_activations = np.loadtxt(activation_file)

n_steps = muscle_activations.shape[0]
num_joints = len(model.dof_jntid)
attempted_qpos = np.zeros((n_steps, num_joints))
attempted_qvel = np.zeros((n_steps, num_joints))
attempted_qacc = np.zeros((n_steps, num_joints))
muscle_velocities = np.zeros((n_steps, model.nu))
muscle_lengths = np.zeros((n_steps, model.nu))
idx = 0
wait_time = 0.5

idx = 0
wait_time = 0.5

with mujoco.viewer.launch_passive(model, data, show_left_ui=False, show_right_ui=True) as viewer:
        
    if model.ncam > 0:
        viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
        viewer.cam.fixedcamid = 0
        mujoco.mj_resetData(model, data)

    for idx in range(n_steps):
        step_start = time.time()
        data.ctrl[:] = muscle_activations[idx,:] * 10
        mujoco.mj_step(model, data)
        attempted_qpos[idx,:] = data.qpos
        attempted_qvel[idx,:] = data.qvel
        attempted_qacc[idx,:] = data.qacc
        muscle_velocities[idx,:] = data.actuator_velocity
        muscle_lengths[idx,:] = data.actuator_length


        with viewer.lock():
            viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time)
            
        viewer.sync()

        time_until_next_step = model.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)

In [ ]:
# plot_functions.plot_joint_state(attempted_qpos)
# plot muscle velocity

plot_functions.plot_muscle_activations(muscle_activations)

In [ ]:
plot_functions.plot_muscle_activations(muscle_velocities)

### Run simulation with quadruped on the ground model

In [320]:
path_to_model = "../Working_Folder/tendon_quadruped_ws_onground.xml"
activation_folder = "../activation_data_files" 

model = mujoco.MjModel.from_xml_path(path_to_model)
data = mujoco.MjData(model)

omega = 1.2
activation_file = f"{activation_folder}/activation_{omega}.txt"
muscle_activations = np.loadtxt(activation_file)

n_steps = muscle_activations.shape[0]
num_joints = len(model.dof_jntid)
attempted_qpos = np.zeros((n_steps, num_joints))
attempted_qvel = np.zeros((n_steps, num_joints))
attempted_qacc = np.zeros((n_steps, num_joints))
idx = 0
wait_time = 0.5


idx = 0
wait_time = 0.5

muscle_velocities = np.zeros((n_steps, model.nu))
muscle_lengths = np.zeros((n_steps, model.nu))

with mujoco.viewer.launch_passive(model, data, show_left_ui=False, show_right_ui=True) as viewer:

    if model.ncam > 0:
        viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
        viewer.cam.fixedcamid = 0
        mujoco.mj_resetData(model, data)

    for idx in range(n_steps):
        step_start = time.time()
        data.ctrl[:] = muscle_activations[idx,:] * 10
        mujoco.mj_step(model, data)
        attempted_qpos[idx,:] = data.qpos
        attempted_qvel[idx,:] = data.qvel
        attempted_qacc[idx,:] = data.qacc
        muscle_velocities[idx,:] = data.actuator_velocity
        muscle_lengths[idx,:] = data.actuator_length

        with viewer.lock():
            viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time)
            
        viewer.sync()
    

        time_until_next_step = model.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)

        # save muscle velocity


In [ ]:
plot_functions.plot_muscle_activations(muscle_lengths)

In [ ]:
# plot muscle velocity

plot_functions.plot_muscle_activations(muscle_velocities)

# Closed-loop simulation with spindle_feedback

### Beta-MN : Skeleto-fusimotor system

In [ ]:
path_to_model = "../Working_Folder/tendon_quadruped_ws_onground.xml"
activation_folder = "../activation_data_files" 

model = mujoco.MjModel.from_xml_path(path_to_model)
data = mujoco.MjData(model)

omega = 1.2
activation_file = f"{activation_folder}/activation_{omega}.txt"
muscle_activations = np.loadtxt(activation_file)

n_steps = muscle_activations.shape[0]
num_joints = len(model.dof_jntid)
attempted_qpos = np.zeros((n_steps, num_joints))
attempted_qvel = np.zeros((n_steps, num_joints))
attempted_qacc = np.zeros((n_steps, num_joints))


muscle_velocities = np.zeros((n_steps, model.nu))
muscle_lengths = np.zeros((n_steps, model.nu))

sensor_names = ['rb_sensor','rf_sensor','lb_sensor','lf_sensor']
leg_muscle_ids = {
    'rb_sensor': [0, 1, 2],  # Right Back leg muscles
    'rf_sensor': [3, 4, 5],  # Right Front leg muscles
    'lb_sensor': [6, 7, 8],  # Left Back leg muscles
    'lf_sensor': [9, 10, 11]  # Left Front leg muscles
}

k_ia = 0.0
k_alpha = 10.0
num_muscles = model.nu 
Ia_per_leg = np.zeros(num_muscles)  


with mujoco.viewer.launch_passive(model, data, show_left_ui=False, show_right_ui=True) as viewer:

    if model.ncam > 0:
        viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
        viewer.cam.fixedcamid = 0
        mujoco.mj_resetData(model, data)

    for idx in range(n_steps):
        step_start = time.time()


        if k_ia!= 0:

            beta_drive = (muscle_activations[idx, :] * k_alpha + Ia_per_leg) * k_ia

            for sensor_name in sensor_names:
                contact_force = data.sensordata[sensor_names.index(sensor_name)] > 0.0  # Check for contact

                if contact_force:  # If there is contact for this leg
                    muscle_ids = leg_muscle_ids[sensor_name]  # Get the muscle IDs for this leg

                    for muscle_id in muscle_ids:
                        muscle_velocity = data.actuator_velocity[muscle_id]  # Get the velocity of the muscle

                        if muscle_velocity > 0:  # If the muscle is lengthening
                            Ia_per_leg[muscle_id] = beta_drive[muscle_id] * muscle_velocity  # Compute Ia feedback
                        else:  # If the muscle is shortening
                            Ia_per_leg[muscle_id] = 0  # No Ia feedback
        else:
            beta_drive = muscle_activations[idx, :] * k_alpha

        data.ctrl[:] = beta_drive
        mujoco.mj_step(model,data)
        
        
        attempted_qpos[idx,:] = data.qpos
        attempted_qvel[idx,:] = data.qvel
        attempted_qacc[idx,:] = data.qacc
        muscle_velocities[idx,:] = data.actuator_velocity
        muscle_lengths[idx,:] = data.actuator_length

        with viewer.lock():
            viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time)
            
        viewer.sync()
    

        time_until_next_step = model.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)

        # save muscle velocity

### Fusimotor system with alpha-to-gamma collateral

In [314]:
path_to_model = "../Working_Folder/tendon_quadruped_ws_onground.xml"
activation_folder = "../activation_data_files" 

model = mujoco.MjModel.from_xml_path(path_to_model)
data = mujoco.MjData(model)

omega = 1.2
activation_file = f"{activation_folder}/activation_{omega}.txt"
muscle_activations = np.loadtxt(activation_file)

n_steps = muscle_activations.shape[0]
num_joints = len(model.dof_jntid)
attempted_qpos = np.zeros((n_steps, num_joints))
attempted_qvel = np.zeros((n_steps, num_joints))
attempted_qacc = np.zeros((n_steps, num_joints))
idx = 0
wait_time = 0.5


idx = 0
wait_time = 1.2

muscle_velocities = np.zeros((n_steps, model.nu))
muscle_lengths = np.zeros((n_steps, model.nu))

sensor_names = ['rb_sensor','rf_sensor','lb_sensor','lf_sensor']
leg_muscle_ids = {
    'rb_sensor': [0, 1, 2],  # Right Back leg muscles
    'rf_sensor': [3, 4, 5],  # Right Front leg muscles
    'lb_sensor': [6, 7, 8],  # Left Back leg muscles
    'lf_sensor': [9, 10, 11]  # Left Front leg muscles
}

k_ia = 0.0
k_alpha = 50
num_muscles = model.nu 
Ia_per_leg = np.zeros(num_muscles)  


with mujoco.viewer.launch_passive(model, data, show_left_ui=False, show_right_ui=True) as viewer:

    if model.ncam > 0:
        viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
        viewer.cam.fixedcamid = 0
        mujoco.mj_resetData(model, data)

    for idx in range(n_steps):
        step_start = time.time()


        if k_ia!= 0:

            alpha_drive = muscle_activations[idx, :] * k_alpha + Ia_per_leg

            for sensor_name in sensor_names:
                contact_force = data.sensordata[sensor_names.index(sensor_name)] > 0.0  # Check for contact

                if contact_force:  # If there is contact for this leg
                    muscle_ids = leg_muscle_ids[sensor_name]  # Get the muscle IDs for this leg

                    for muscle_id in muscle_ids:
                        muscle_velocity = data.actuator_velocity[muscle_id]  # Get the velocity of the muscle

                        if muscle_velocity > 0:  # If the muscle is lengthening
                            Ia_per_leg[muscle_id] = (alpha_drive[muscle_id] * k_ia) * muscle_velocity  # Compute Ia feedback
                            # Ia_per_leg[muscle_id] = (k_ia) * muscle_velocity
                        else:  # If the muscle is shortening
                            Ia_per_leg[muscle_id] = 0  # No Ia feedback
        else:
            alpha_drive = muscle_activations[idx, :] *  k_alpha

        data.ctrl[:] = alpha_drive
        mujoco.mj_step(model,data)
        
        
        attempted_qpos[idx,:] = data.qpos
        attempted_qvel[idx,:] = data.qvel
        attempted_qacc[idx,:] = data.qacc
        muscle_velocities[idx,:] = data.actuator_velocity
        muscle_lengths[idx,:] = data.actuator_length

        with viewer.lock():
            viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = int(data.time)
            
        viewer.sync()
    

        time_until_next_step = model.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)
